In [1]:
# import the data
# import sentiment_idx.xlsd as pandas
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
import numpy as np

# import the data
sentiment = pd.read_csv('data/sentiment_idx.csv')
pe = pd.read_csv('data/spy_pe.csv')
sp = pd.read_csv('data/sp500.csv')

In [3]:
# create datetime column for each dataframe
sentiment['date'] = pd.to_datetime(sentiment['date'])
pe['date'] = pd.to_datetime(pe['date'])
sp['date'] = pd.to_datetime(sp['date'])
# set index to datetime
sentiment.set_index('date', inplace=True)
pe.set_index('date', inplace=True)
sp.set_index('date', inplace=True)

In [4]:
# merge sentiment and sp
df = pd.merge(sentiment, sp, how='inner', on='date')
# resample pe to daily and interpolate
pe = pe.resample('D').asfreq()
pe['pe'] = pe['pe'].interpolate(method='spline', order=2)
# left merge df and pe
df = pd.merge(df, pe, how='inner', on='date')
df.dropna(inplace=True)
df.drop(['open', 'high', 'low', 'close', 'volume'], axis=1, inplace=True)
# rename adj close to price
df.rename(columns={'adj close': 'price'}, inplace=True)

In [58]:
# calculate percent change
df['pct_change'] = df['price'].pct_change()
# create null countdown column
df['countdown'] = np.nan
# for all rows where percent_change is less than -5%, set countdown to 0
df.loc[df['pct_change'] < -0.05, ['countdown']] = 0

In [59]:
# get all rows where countdown is 0
df[df['countdown'] == 0]

,sentiment,price,pe,pct_change,countdown
date,,,,,
1987-10-16,0.169563,282.700012,16.156877,-0.051597,0.0
1987-10-19,0.132298,224.839996,16.126337,-0.204669,0.0
1987-10-26,-0.010434,227.669998,16.053110,-0.082789,0.0
1988-01-08,-0.084208,243.399994,15.112685,-0.067683,0.0
1989-10-13,0.126801,333.649994,12.711512,-0.061172,0.0
1997-10-27,0.243113,876.989990,23.510110,-0.068657,0.0
1998-08-31,-0.044410,957.280029,31.979911,-0.068014,0.0
2000-04-14,0.244439,1356.560059,31.886089,-0.058278,0.0
2008-09-29,-0.368281,1106.420044,73.715587,-0.088068,0.0


In [60]:
# get the number of rows where countdown is nan
df['countdown'].isna().sum()

10261

In [61]:
for i in range(len(df)-1, 0, -1):
    if np.isnan(df.loc[df.index[i], 'countdown']):
        continue
    if df.loc[df.index[i], 'countdown'] == 0:
        df.loc[df.index[i-1], 'countdown'] = 1
    # create condition for if previous row is 0
    # elif df.loc[df.index[i-1], 'countdown'] == 0:
    #     continue
    else:
        df.loc[df.index[i-1], 'countdown'] = df.loc[df.index[i], 'countdown'] + 1

In [62]:
df[df['countdown']==0]

,sentiment,price,pe,pct_change,countdown
date,,,,,
2020-06-11,-0.607016,3002.100098,22.805288,-0.058944,0.0


In [63]:
# drop na rows
df.dropna(inplace=True)

In [64]:
# downsample price to weekly with moving average
df['price_smooth'] = df['price'].rolling(25).mean()

In [65]:
# plot price_smooth
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['price_smooth'], name='price_smooth'))
fig.update_layout(title='Price Smooth', xaxis_title='Date', yaxis_title='Price')

In [107]:
# create column y such that y = 1 if price(t) > price(t-1), y = -1 if price(t) < price(t-1) and y = 0 otherwise
diff = df['price_smooth'].diff()

# create a new column y based on the difference
df['y'] = np.where(diff > 2, 1, np.where(diff < -2, -1, 0))

In [108]:
# get correlation between y and sentiment
df[['y', 'sentiment']].corr()

,y,sentiment
y,1.000000,0.143198
sentiment,0.143198,1.000000


In [92]:
# get correlation between y and pe
df[['y', 'pe']].corr()

,y,pe
y,1.000000,-0.124206
pe,-0.124206,1.000000


In [79]:
# define the function to optimize (i.e., the given equation)
def f(x, a, b, c, d, j):
    # return np.tanh(a*np.exp(-(x[:,0]-b)) + c*np.exp(-(x[:,1]-d))) + j
    return np.tanh(a * np.square(x[:,0] - b) + c * np.square(x[:,1] - d)) + j

# load data from dataframe
X = df[['price', 'pe']].values
y = df['y'].values

# fit the function to the data using curve_fit
popt, pcov = curve_fit(f, X, y)

# extract the optimized parameters
a, b, c, d, j = popt

# calculate the predicted values
df['y_pred'] = basis(X, a, b, c, d, j)

# calculate the mean squared error between predicted and true values
mse = np.mean((y - df['y_pred'])**2)
pcov

/Users/johnboesen/Documents/Code/#Algotraders/yja/yja-env/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning:

Covariance of the parameters could not be estimated



array([[inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf]])

In [78]:
# plot y_pred 
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['y_pred'], name='y_pred'))
fig.show()

# Testing
Create synthetic securities with the correlation we want to test with

In [54]:
# def generate_random_returns(length):
#     return np.random.normal(0, 1, length)

# def scale_returns(returns, target_std):
#     std = np.std(returns)
#     return returns * target_std / std

# import time

# def model_security_with_timer(correlation):
#     length = len(df['price'])
#     spy_std = np.std(df['price'])
#     target_std = spy_std / np.sqrt(correlation)
#     start_time = time.time()  # record the start time
#     while True:
#         s_returns = generate_random_returns(length)
#         scaled_s_returns = scale_returns(s_returns, target_std)
#         actual_correlation = np.corrcoef(df['price'], scaled_s_returns)[0, 1]
#         if np.abs(actual_correlation - correlation) < 0.01:
#             break
#         elapsed_time = time.time() - start_time  # calculate elapsed time
#         if elapsed_time > 2:
#             print(np.corrcoef(df['price'], scaled_s_returns))
#             print(f'i: {correlation}, actual correlation: {actual_correlation}')
#             start_time = time.time()  # reset start time
#     return pd.Series(scaled_s_returns, index=df['price'].index)

# # create 20 columns with correaltions from [-1, 1]
# # for i in range(-10, 11):
# #     df[f'{i/10}r'] = model_security_with_timer(i / 10)
# model_security_with_timer(0.7)

[[1.         0.01050421]
 [0.01050421 1.        ]]
i: 0.7, actual correlation: 0.010504210194910601
[[ 1.         -0.00164048]
 [-0.00164048  1.        ]]
i: 0.7, actual correlation: -0.0016404844847845426
[[1.         0.00413701]
 [0.00413701 1.        ]]
i: 0.7, actual correlation: 0.004137006306044751
[[ 1.         -0.00600583]
 [-0.00600583  1.        ]]
i: 0.7, actual correlation: -0.0060058278843573305
[[ 1.         -0.00489324]
 [-0.00489324  1.        ]]
i: 0.7, actual correlation: -0.004893238252067178
[[ 1.         -0.00502864]
 [-0.00502864  1.        ]]
i: 0.7, actual correlation: -0.005028638006457374
[[1.         0.00352456]
 [0.00352456 1.        ]]
i: 0.7, actual correlation: 0.003524557329324017
[[1.         0.01255669]
 [0.01255669 1.        ]]
i: 0.7, actual correlation: 0.012556693706623446
[[1.         0.02919492]
 [0.02919492 1.        ]]
i: 0.7, actual correlation: 0.02919491698042917
[[1.         0.02324175]
 [0.02324175 1.        ]]
i: 0.7, actual correlation: 

KeyboardInterrupt: 